In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "NF"
BARK_VAR = "WC"
FIA_SPCD = 22

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.0, 0.904973, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,1114.000000,1.114000e+03,1114.0,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.000000,...,49.000000,49.000000,1114.000000,1114.000000,1114.000000,1114.000000,1114.0,1114.000000,1114.000000,1114.000000
mean,616.934470,4.647665e+11,22.0,137.366457,0.385745,3.252235,40.093357,15.732496,0.613743,0.536831,...,0.480612,1.040816,2005.359964,8.143627,2015.359964,7.667864,10.0,1.964093,6.345601,122.928187
std,29.048248,6.362469e+10,0.0,47.788318,0.229333,1.737888,7.877789,12.008833,0.249002,0.221837,...,0.346232,0.720147,2.268313,1.330079,2.268313,1.457733,0.0,0.498483,4.687536,69.151436
min,505.000000,6.020935e+10,22.0,10.000000,0.000000,0.000000,5.000000,5.000000,0.060000,0.010000,...,0.050000,0.050000,2001.000000,4.000000,2011.000000,4.000000,10.0,0.000000,0.000000,0.000000
25%,603.000000,4.101040e+11,22.0,102.500000,0.180000,1.710423,35.000000,6.900000,0.372500,0.350000,...,0.200000,0.500000,2004.000000,7.000000,2014.000000,7.000000,10.0,2.000000,2.000000,67.000000
50%,606.000000,4.103066e+11,22.0,142.500000,0.365000,3.665191,39.000000,9.900000,0.650000,0.500000,...,0.400000,0.850000,2005.000000,8.000000,2015.000000,8.000000,10.0,2.000000,7.000000,121.000000
75%,618.000000,5.307042e+11,22.0,172.400000,0.560000,4.638212,45.000000,23.200000,0.850000,0.750000,...,0.700000,1.500000,2007.000000,9.000000,2017.000000,9.000000,10.0,2.000000,12.000000,176.000000
max,800.000000,5.309052e+11,22.0,274.704527,1.550000,6.283185,65.000000,76.500000,0.990000,0.980000,...,1.450000,2.650000,2009.000000,11.000000,2019.000000,11.000000,10.0,7.000000,16.000000,251.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [07:31<00:00,  4.43it/s, 127 steps of size 4.87e-02. a


Done.
Samples saved at ../../models/maicf/NF//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/NF//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [14:47<00:00,  2.25it/s, 127 steps of size 2.97e-02. a


Done.
Samples saved at ../../models/maicf/NF//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/NF//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/NF//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/NF//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [07:34<00:00,  4.40it/s, 127 steps of size 4.48e-02. a


Done.
Samples saved at ../../models/maicf/NF//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/NF//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [15:26<00:00,  2.16it/s, 127 steps of size 3.08e-02. a


Done.
Samples saved at ../../models/maicf/NF//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/NF//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [06:08<00:00,  5.43it/s, 127 steps of size 4.25e-02. a


Done.
Samples saved at ../../models/maicf/NF//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/NF//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [05:52<00:00,  5.67it/s, 127 steps of size 5.01e-02. a


Done.
Samples saved at ../../models/maicf/NF//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/NF//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/NF//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/NF//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [11:03<00:00,  3.02it/s, 127 steps of size 2.83e-02. a


Done.
Samples saved at ../../models/maicf/NF//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/NF//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [11:05<00:00,  3.00it/s, 127 steps of size 3.04e-02. a


Done.
Samples saved at ../../models/maicf/NF//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/NF//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
